In [1]:
print("""This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports""")

This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import json
from copy import deepcopy

In [3]:
RUN_BASELINE_ONLY = False
image_mode = False
debug_mode = False

drop_percentages = [0.1, 0.2, 0.27, 0.35, 0.43 , 0.5]
#drop_percentages = [0.2, 0.35, 0.5, 0.65, 0.75]
number_of_generated_subgraphs = 10
baseline_file = "experiment_4"

In [4]:
results = {}
nodes_results = {}
total_nodes_dict = {}
avg_total_nodes_results = {}
baseline_success = None

In [5]:
# iterator initiation
i = 0
while i < len(drop_percentages):
    drop_pct = drop_percentages[i]

    import logging
    #  loop over copy
    handlers_list = logging.root.handlers[:]
    h_idx = 0
    while h_idx < len(handlers_list):
        handler = handlers_list[h_idx]
        handler.close(); logging.root.removeHandler(handler) 
        h_idx = h_idx + 1
    
    # The baseline does not change since there are no nodes dropped
    if i == 1:
        #  concatinating 
        base_log = baseline_file + '.log'
        if os.path.exists(base_log) == True:
            #  open/close
            f = open(base_log, 'r')
            content = f.read()
            f.close()
            
            baseline_match = re.search(r'Attacker can guarantee success probability of: ([0-9.]+)', content)
            
            #  checking 
            if baseline_match != None:
                val_str = baseline_match.group(1)
                baseline_success = float(val_str)
            
            print(f"Baseline attacker success: {baseline_success}")

    print(f"\n{'='*50}")
    print(f"RUNNING EXPERIMENT WITH DROP PERCENTAGE: {drop_pct*100}%")
    print(f"{'='*50}\n")
    
    # experiment_name = f"experiment_drop_{int(drop_pct*100)}"
    pct_val = int(drop_pct*100)
    experiment_name = "experiment_drop_" + str(pct_val)
    
    # Run the experiment notebook
    %run "{baseline_file}.ipynb"
    
    # subgraph_file = f"sub_{baseline_file}.log"
    sub_name = "sub_" + baseline_file + ".log"
    subgraph_file = sub_name
    
    if os.path.exists(subgraph_file) == True:
        #  read
        f = open(subgraph_file, 'r')
        content = f.read()
        f.close()
        
        avg_match = re.search(r'Average attacker success across subgraphs: ([0-9.]+)', content)
        nodes_match = re.search(r'Initial number of dropped nodes per subgraph: ([0-9]+)', content)
        avg_total_nodes_match = re.search(r'Average total nodes dropped \(including cleanup\): ([0-9.]+)', content)
        total_nodes_match = re.search(r'Total number of nodes in the graph: ([0-9]+)', content)

        if avg_match != None:
            g1 = avg_match.group(1)
            avg_success = float(g1)
            results[drop_pct] = round(avg_success, 3)
            
            # manual diff logic
            diff = "N/A"
            if baseline_success:
                diff = avg_success - baseline_success
                
            print(f"Drop %: {drop_pct*100}%, Avg Success: {avg_success}, Diff: {diff}")

        if nodes_match != None:
            g2 = nodes_match.group(1)
            dropped_nodes = int(g2)
            nodes_results[drop_pct] = dropped_nodes
            print(f"Number of dropped nodes per subgraph: {dropped_nodes}")

        # nested if instead of 'and'
        if avg_total_nodes_match != None:
            if total_nodes_match != None:
                v_avg = avg_total_nodes_match.group(1)
                v_tot = total_nodes_match.group(1)
                
                avg_total_nodes = float(v_avg)
                total_nodes = int(v_tot)
                
                avg_total_nodes_results[drop_pct] = avg_total_nodes
                total_nodes_dict[drop_pct] = total_nodes  

    else:
        print(f"Warning: Could not find subgraph log file")
        
    i = i + 1


RUNNING EXPERIMENT WITH DROP PERCENTAGE: 10.0%

This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports
Current settings: image_mode=False, debug_mode=False, RUN_BASELINE_ONLY=False
We start with the baseline graph calculation!
Now we are going to run the subgraph analysis
The subgraph analysis is done!
[1] "experiment_4.log"
[1] "Tue Dec 30 20:07:41 2025"



BASELINE RUN: BOTH ATTACKER AND DEFENDER HAVE FULL GRAPH KNOWLEDGE



Debug - Current Attack paths:
  0: [0, 1, 5, 15, 'c(12,13,14,16)']
  1: [0, 3, 6, 8, 'c(12,13,14,16)']
  2: [0, 3, 6, 8, 10, 15, 'c(12,13,14,16)']
  3: [0, 3, 8, 'c(12,13,14,16)']
  4: [0, 3, 8, 10, 15, 'c(12,13,14,16)']
  5: [0, 4, 7, 'c(12,13,14,16)']
  6: [0, 4, 7, 10, 15, 'c(12,13,14,16)']
  7: [0, 2, 'c(12,13,14,16)']
  8: [0, 2, 9, 'c(12,13,14,16)']
  9: [0, 2, 10, 15, 'c(12,13,14,16)']
  10: [0, 2, 11, 'c(12,13,14,16)']

+++++++++++++++++++++++++++++

The virtual target nodeid

In [6]:
# Create DataFrame
df = pd.DataFrame({
    'Drop Percentage': [k*100 for k in results.keys()],
    'Dropped Nodes': [nodes_results.get(k, float('nan')) for k in results.keys()],
    'Avg Total Nodes Dropped': [avg_total_nodes_results.get(k, float('nan')) for k in results.keys()],
    'Actual Drop Percentage': [avg_total_nodes_results.get(k, float('nan'))/total_nodes_dict.get(k, 17)*100 for k in results.keys()],
    'Average Success': list(results.values()),
    'Baseline': [baseline_success] * len(results),
    'Difference': [results[k] - baseline_success for k in results.keys()]
})

# Print dataframe here to read
# Store it in some external .csv file for later use
print("\nFinal Results:")
print(df)
df.to_csv(f'{baseline_file}_extrapolation_results.csv', index=False)


Final Results:
   Drop Percentage  Dropped Nodes  Avg Total Nodes Dropped  \
0             10.0              1                      2.1   
1             20.0              2                      2.7   
2             27.0              3                      4.1   
3             35.0              4                      5.9   
4             43.0              5                      7.7   
5             50.0              6                      8.0   

   Actual Drop Percentage  Average Success  Baseline  Difference  
0               12.352941            0.034     0.025       0.009  
1               15.882353            0.032     0.025       0.007  
2               24.117647            0.036     0.025       0.011  
3               34.705882            0.038     0.025       0.013  
4               45.294118            0.040     0.025       0.015  
5               47.058824            0.042     0.025       0.017  
